In [ ]:
from importlib import reload
#---------------------------------------------------------------------
import sys, os
import pandas as pd
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
from DOVSOutages import DOVSOutages
from DABatch import DABatchOPCO, DABatch
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities

In [ ]:
#--------------------------------------------------
# In general, only date_0 and date_1 will need to be changed!
#--------------------------------------------------
date_0 = '2025-05-18'
date_1 = '2025-05-24'
#-------------------------
# NOTE: Typically 'ky' and 'tx' are excluded due to lack of data.
#       For 'ky' this is likely due to a lack of AMI meters I suppose.
#       For 'tx' this is likely due to the funny way they handle their data
#       This can and should be reassessed later
opcos = ['ap', 'im', 'oh', 'pso', 'swp']
# If one wants to select specific states from opcos, do something similar to the following, which
#   will select only states ['TN', 'VA'] from opco 'ap', only state 'OH' from opco 'oh', and 
#   all states included in opcos 'im', 'pso', and 'swp'
# opcos = dict(ap=['TN', 'VA'], im=None, oh='OH', pso=None, swp=None)
#-------------------------
# If save_dir_base is None, os.path.join(Utilities.get_local_data_dir(), 'dovs_check') will be used
#     For me, this is r'C:\Users\s346557\Documents\LocalData\dovs_check'
#-----
# If, e.g., date_0='2025-01-12', date_1='2025-01-18', dates_subdir_appndx=None the results for opco='ap' will be found in
#     r'C:\Users\s346557\Documents\LocalData\dovs_check\20250112_20250118\ap\'
# If, e.g., dates_subdir_appndx = '_blah', this will be altered to 
#     r'C:\Users\s346557\Documents\LocalData\dovs_check\20250112_20250118_blah\ap\'
#-----
save_dir_base                      = None
dates_subdir_appndx                = None
#-------------------------
CI_NB_min                          = None # 15 was old cut, now opened flood gates and accept all
mjr_mnr_cause                      = None
use_sql_std_outage                 = True 
addtnl_outg_sql_kwargs             = None
#-------------------------
daq_search_time_window             = pd.Timedelta('24 hours')
outg_rec_nbs                       = None
#-------------------------
# DOVSAudit arguments
dovs_audit_args                    = dict(
    calculate_by_PN                   = True, 
    combine_by_PN_likeness_thresh     = pd.Timedelta('15 minutes'),  
    expand_outg_search_time_tight     = pd.Timedelta('1 hours'), 
    expand_outg_search_time_loose     = pd.Timedelta('12 hours'), 
    use_est_outg_times                = False, 
    use_full_ede_outgs                = False, 
    overlaps_addtnl_dovs_sql_kwargs   = dict(
        CI_NB_min  = 0, 
        CMI_NB_min = 0
    ), 
)
#-------------------------
load_prereqs_if_exist              = True
reanalyze_preex_results            = False
perform_plotting                   = True
build_summary_dfs                  = True
#-------------------------
merge_outputs                      = True
output_subdir                      = 'AllOPCOs'
#-------------------------
rebuild_outg_rec_nb_to_files_dicts = False
#-------------------------
run_outg_inclusion_assessment      = True
max_pct_PNs_missing_allowed        = 0
n_PNs_w_power_threshold            = 95
include_suboutg_endpt_plots        = True
#-------------------------
debug                              = False
verbose                            = True
#-------------------------
run_daq                            = True
analyze_audits                     = True

In [ ]:
# NOTE: For Mico's No Interruptions request, the following changes were made
# use_sql_std_outage                 = False 
# addtnl_outg_sql_kwargs             = dict(
#     MJR_CAUSE_CD = 'NI'
# )
# dovs_audit_args                    = dict(
#     calculate_by_PN                   = True, 
#     combine_by_PN_likeness_thresh     = pd.Timedelta('15 minutes'),  
#     expand_outg_search_time_tight     = pd.Timedelta('1 hours'), 
#     expand_outg_search_time_loose     = pd.Timedelta('12 hours'), 
#     use_est_outg_times                = False, 
#     use_full_ede_outgs                = False, 
#     overlaps_addtnl_dovs_sql_kwargs   = dict(
#         CI_NB_min  = None, 
#         CMI_NB_min = None
#     ), 
# )

In [ ]:
#--------------------------------------------------
da_batch = DABatch(
    date_0                      = date_0, 
    date_1                      = date_1, 
    opcos                       = opcos, 
    save_dir_base               = save_dir_base, 
    dates_subdir_appndx         = dates_subdir_appndx, 
    CI_NB_min                   = CI_NB_min, 
    mjr_mnr_cause               = mjr_mnr_cause, 
    use_sql_std_outage          = use_sql_std_outage, 
    addtnl_outg_sql_kwargs      = addtnl_outg_sql_kwargs, 
    daq_search_time_window      = daq_search_time_window, 
    outg_rec_nbs                = outg_rec_nbs, 
    dovs_audit_args             = dovs_audit_args, 
)
#--------------------------------------------------
if run_daq:
    da_batch.run_batch_daq(
        load_prereqs_if_exist = load_prereqs_if_exist, 
        batch_size_ami        = 25, 
        n_update_ami          = 1, 
        batch_size_ede        = 25, 
        n_update_ede          = 1, 
        pdpu_only_ede         = True, 
        verbose               = True
    )
#--------------------------------------------------
if analyze_audits:
    results = da_batch.analyze_audits_for_opcos_from_csvs(
        reanalyze_preex_results            = reanalyze_preex_results, 
        perform_plotting                   = perform_plotting, 
        build_summary_dfs                  = build_summary_dfs, 
        #-----
        merge_outputs                      = merge_outputs, 
        output_subdir                      = output_subdir, 
        #-----
        rebuild_outg_rec_nb_to_files_dicts = rebuild_outg_rec_nb_to_files_dicts, 
        #-----
        run_outg_inclusion_assessment      = run_outg_inclusion_assessment, 
        max_pct_PNs_missing_allowed        = max_pct_PNs_missing_allowed, 
        n_PNs_w_power_threshold            = n_PNs_w_power_threshold, 
        include_suboutg_endpt_plots        = include_suboutg_endpt_plots, 
        fig_num                            = 0, 
        #-----
        verbose                            = verbose, 
        debug                              = debug
    )